In [1]:
from sage.algebras.flag_algebras import *

In [72]:
sys.setprofile(None)

def tracefunc(frame, event, arg, indent=[0]):
    if event == "call":
        indent[0] += 2
        module_name = inspect.getmodule(frame.f_code).__name__ if inspect.getmodule(frame.f_code) else "unknown"
        class_name = ""
        try:
            class_name = inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True)[-1][0][0].__name__
        except:
            if 'self' in frame.f_locals:
                class_name = str(inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True))
            else:
                class_name = ""
        function_name = frame.f_code.co_name
        tst = module_name + function_name
        if "IPython" in tst or "enum" in tst or "typing" in tst or "traitlets" in tst or "util" in tst or "ipykernel" in tst:
            pass
        else:
            if "unknown" in tst or "tokenize" in tst or "logging" in tst or "json" in tst or "preparse" in tst or indent[0]<=0:
                pass
            else:
                print("-" * indent[0] + "> call function", function_name, "in", class_name, "from", module_name)
    elif event == "return":
        indent[0] -= 2
    return tracefunc

#sys.setprofile(tracefunc)

#asd = sum([G(0)])

In [79]:
loglevel = 2

def set_sum(self):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    repl = {gs[-1]: 1 - sum(gs[:-1])}
    nvec = vector([xx.subs(repl) for xx in valvec])
    return self.parent()(self.size(), nvec)

def subs(self, args):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    repl = {gs[ii]:args[ii] for ii in range(min(len(args), len(gs)))}
    nvec = vector([xx.subs(repl) for xx in valvec])
    retalg = FlagAlgebra(QQ, self.parent().theory())
    return retalg(self.size(), nvec)

def derivative(self, times):
    valvec = self.values()
    if len(valvec)==0:
        return self
    par = valvec[0].parent()
    gs = par.gens()
    rvec = []
    for ff, vv in enumerate(valvec):
        if vv==0:
            rvec.append(0)
            continue
        aval = vv
        for ii in range(min(len(times), len(gs))):
            aval = aval.derivative(gs[ii], times[ii])
        rvec.append(aval)
    return self.parent()(self.size(), vector(rvec))

def derivatives(self, point):
    times = []
    for xx in self.values():
        if xx==0:
            continue
        dd = xx.dict()
        for kk in dd.keys():
            kk = tuple(kk)
            if kk in times:
                continue
            for ll in itertools.product(*[range(ii+1) for ii in kk]):
                if ll not in times:
                    times.append(ll)
    res = []
    if loglevel>1:
        print(times)
    for xx in times:
        der = subs(derivative(self, xx), point)
        minnz = 1000000
        for xx in der.values():
            if int(xx)!=0:
                minnz = min(abs(xx), minnz)
        if minnz != 1000000:
            res.append(der/minnz)
    return res

def blowup_construction(self, target_size, construction_size, variable=False, symmetric=False, unordered=False, **kwargs):
    from tqdm import tqdm
    from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
    R = PolynomialRing(QQ, construction_size, "X")
    gs = R.gens()
    res = 0

    if symmetric:
        terms = ((sum(gs))**target_size).dict()
        if loglevel>0:
            iterator = tqdm(terms)
        else:
            itertor = terms
        for exps in iterator:
            verts = []
            for ind, exp in enumerate(exps):
                verts += [ind]*exp
            coeff = terms[exps]/(construction_size**target_size)
            if variable:
                coeff = terms[exps]
                for ind, exp in enumerate(exps):
                    coeff *= gs[ind]**exp
            blocks = {}
            for rel in kwargs:
                if rel not in self.signature():
                    continue
                reledges = kwargs[rel]
                bladd = []
                for edge in reledges:
                    clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                    bladd += list(set([tuple(sorted(xx)) for xx in itertools.product(*clusters) if len(set(xx))==len(edge)]))
                blocks[rel] = bladd
            res += self(target_size, **blocks).afae()*coeff
    else:
        rep = int(target_size if not unordered else target_size - 1)
        if loglevel>0:
            iterator = tqdm(itertools.product(range(construction_size), repeat=rep))
        else:
            iterator = itertools.product(range(construction_size), repeat=rep)
        for verts in iterator:
            if unordered:
                verts = [0] + list(verts)

            coeff = 1/(construction_size**rep)
            if variable:
                coeff = 1
                for ind in verts:
                    coeff *= gs[ind]
            
            blocks = {}
            for rel in kwargs:
                if rel not in self.signature():
                    continue
                reledges = kwargs[rel]
                bladd = []
                for edge in reledges:
                    clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                    bladd += list(set([tuple(sorted(xx)) for xx in itertools.product(*clusters) if len(set(xx))==len(edge)]))
                blocks[rel] = bladd
            res += self(target_size, **blocks).afae() * coeff
    return res

def sym_asym_bases(self, n, ftype=None):
    r"""
    Generate the change of base matrices for the symmetric
    and the asymmetric subspaces
    """
    
    flags = self.generate_flags(n, ftype)
    uniques = []
    sym_base = []
    asym_base = []
    for xx in flags:
        xxid = self.identify(n, [xx.ftype()], **xx.blocks())
        if xxid not in uniques:
            uniques.append(xxid)
            sym_base.append(xx.afae())
        else:
            sym_ind = uniques.index(xxid)
            asym_base.append(sym_base[sym_ind] - xx.afae())
            sym_base[sym_ind] += xx
    m_sym = matrix(len(sym_base), len(flags), [xx.values() for xx in sym_base], sparse=True)
    m_asym = matrix(len(asym_base), len(flags), [xx.values() for xx in asym_base], sparse=True)
    return m_sym, m_asym

#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    if loglevel<=0:
        return
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def print_debug(*args):
    if loglevel<=1:
        return
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")

#continued fraction rounding
def cfr(value, quotient_bound=7, denom_bound=9):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, force_pos=False):
    if force_pos:
        return [max(cfr(xx), 0) for xx in ls]
    else:
        return [cfr(xx) for xx in ls]

#rounding for matrices
def cfr_matrix(mat):
    return matrix(QQ, [cfr_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def cfr_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#rounding based on simple denominator scaling
def sdem(value, den=1024):
    return floor(value*den)/den

#rounding for lists
def sdem_list(ls, force_pos=False):
    if force_pos:
        return [max(sdem(xx), 0) for xx in ls]
    else:
        return [sdem(xx) for xx in ls]

#rounding for matrices
def sdem_matrix(mat):
    return matrix(QQ, [sdem_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def sdem_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, sdem_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, sdem_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#adaptive continued fraction rounding, seems to work well
def cfr_adaptive(ls, onevec):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([cfr(xx, resol1, resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec


#helper function, to add the kernel constraints to the table constructor based on a provided phi vector
def adjust_table_phi(self, table_constructor, phi_vectors_exact, test=False):
    
    for param in table_constructor.keys():
        ns, ftype = param
        table = self.mul_project_table(ns, ns, ftype, [])
        Zs = [[None for _ in range(len(phi_vectors_exact))] for _ in range(len(table_constructor[param]))]
        for gg, morig in enumerate(table):
            for ii, base in enumerate(table_constructor[param]):
                mat = base * morig * base.T
                for phind, phi_vector_exact in enumerate(phi_vectors_exact):
                    if Zs[ii][phind]==None:
                        Zs[ii][phind] = mat*phi_vector_exact[gg]
                    else:
                        Zs[ii][phind] += mat*phi_vector_exact[gg]
        
        new_bases = []
        for ii, Zgroup in enumerate(Zs):
            Z = None
            for Zjj in Zgroup:
                if test and min(Zjj.eigenvalues())<0 and loglevel>0:
                    print("Construction based Z matrix for {} is not semidef: {}".format(ftype, min(Zjj.eigenvalues())))
                if Z==None:
                    Z = Zjj
                else:
                    Z.augment(Zjj)
            Zk = Z.kernel()
            print_debug("\n\n", Z, Zk)
            Zkern = Zk.basis_matrix()
            if Zkern.nrows()>0:
                new_bases.append(Zkern * table_constructor[param][ii])
        table_constructor[param] = new_bases

    return table_constructor

def print_eigenvalues(self, table_constructor, sdp_result):
    block_index = 0
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            X_approx = matrix(sdp_result['X'][block_index + plus_index])
            X_eigenvalues = X_approx.eigenvalues()
            print_debug(ftype, plus_index, X_eigenvalues, "\n\n")
            
        block_index += len(table_constructor[params])

#helper function, moves a table to an sdp input data
def tables_to_sdp_data(self, table_constructor, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        block_sizes += [base.nrows() for base in table_constructor[params]]
        
        #only loop through the table once
        for gg, morig in enumerate(table):
            #for each base change create the entries
            for plus_index, base in enumerate(table_constructor[params]):
                mm = base * morig * base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, block_index + plus_index, iinds[cc]+1, jinds[cc]+1])
                            mat_vals.append(values[cc])
        block_index += len(table_constructor[params])
    return mat_inds, mat_vals, block_sizes

#helper function, moves the linear constraints to the sdp input data
def constraints_to_sdp_data(self, flag_num, constraints_vals, constraints_flags_vec, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1

    constr_num = len(constraints_vals)
    
    for ii in range(constr_num):
        mat_inds.extend([0, block_index+1, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    for gg in range(flag_num):
        mat_inds.extend([gg+1, block_index, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(constr_num):
            mat_inds.extend([gg+1, block_index+1, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    block_sizes += [-flag_num, -constr_num]
    
    return mat_inds, mat_vals, block_sizes

#flatten and unflatten symmetric matrices
def flatten_matrix(mat, doubled=False):
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def unflatten_matrix(ls, dim, doubled=False):
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

#round the sdp output based on slacks
def round_sdp_solution(self, sdp_result, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact):
    
    if loglevel>=1:
        print("Begin rounding\n")

    phi_vector_exact = phi_vectors_exact[0]
    
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints
    print_debug(block_sizes, target_vector_exact, phi_vector_exact, positives_matrix_exact)
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(sdem_list(c_vector_approx)) # as above but rounded
    
    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]    

    if loglevel>=2:
        print("\nc values and related\n")
    print_debug(flags_num, c_vector_approx, c_vector_rounded, c_zero_inds, c_nonzero_inds)
    
    
    
    positives_num = -block_sizes[-1] - 2 # same as m, number of positive constraints (-2 for the equality)
    
    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive
    
    e_vector_approx = vector(sdp_result['X'][-1][:-2]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(sdem_list(e_vector_approx)) # as above but rounded
    
    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(positives_num) if ff not in e_zero_inds]

    if loglevel>=2:
        print("\ne values and related\n")
    print_debug(positives_num, phi_pos_vector_exact, e_vector_approx, e_vector_rounded, e_zero_inds, e_nonzero_inds)
    
    
    
    bound_exact = target_vector_exact*phi_vector_exact # the u value, the bound we want to prove
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([target_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, len(e_nonzero_inds), len(c_zero_inds), [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzero_inds]

    if loglevel>=2:
        print("\nrelevant parameters for stuff\n")
    print_debug(bound_exact, corrected_target_relevant_exact, positives_matrix_relevant_exact, e_nonzero_list_rounded)
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 
    
    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += sdem_list(flatten_matrix(X_approx))
            
            M_extra = []
            
            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(flatten_matrix((base * M_FF * base.T).rows(), doubled=True))
            
            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])
    
    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 
    
    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact.T)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)

    # correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact)

    if loglevel>=2:
        print("\nrelevant M and X stuff flattened and correction\n")
    dim_M_flat = M_flat_relevant_matrix_exact.dimensions()
    dim_pos = positives_matrix_relevant_exact.dimensions()
    print_debug(dim_M_flat, dim_pos, M_flat_relevant_matrix_exact, X_flat_vector_rounded, M_matrix_final, x_vector_final, x_vector_corr)
    mtm = (M_matrix_final * M_matrix_final.T)
    print_debug(mtm)
    
    #
    # Recover the X matrices and e vector from the corrected x
    #
    
    e_nonzero_vector_corr = x_vector_corr[-len(e_nonzero_inds):]
    e_vector_corr = vector(QQ, positives_num, dict(zip(e_nonzero_inds, e_nonzero_vector_corr)))
    
    if min(e_vector_corr)<0 and loglevel>=1:
        print("Linear coefficient is negative: {}".format(min(e_vector_corr)))
    
    X_matrix_corr = []
    for ii, block_dim in enumerate(block_sizes):
        if block_dim<0:
            break
        X_matrix_ii_corr, x_vector_corr = unflatten_matrix(x_vector_corr, block_dim)
        X_matrix_corr.append(matrix(X_matrix_ii_corr))
        if min(X_matrix_ii_corr.eigenvalues())<0 and loglevel>=1:
            print("Rounded X matrix is not semidefinite: {}".format(min(X_matrix_ii_corr.eigenvalues())))
    X_matrix_corr.append(e_vector_corr)
    
    
    #
    # Verify the bound and semidefiniteness
    #

    block_index = 0
    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_corr
    
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            X_flat_vector_corr = vector(flatten_matrix(X_matrix_corr[block_index + plus_index].rows()))
            
            for gg, morig in enumerate(table):
                mm = base * morig * base.T
                M_flat_vector_exact = vector(flatten_matrix((base * morig * base.T).rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*X_flat_vector_corr
        
        block_index += len(table_constructor[params])

    if loglevel>=2:
        print("\nfinal_values\n")
    print_debug(e_nonzero_vector_corr, e_vector_corr, positives_matrix_exact.T*e_vector_corr, slacks, X_matrix_corr)
    
    return min(slacks), X_matrix_corr

#the combined code
def optimize_problem(self, target_element, target_size, maximize=True, positives=None, construction=None, certificate=False, rounding=True):
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time
    
    #
    # initial setup
    #
    
    base_flags = self.generate_flags(target_size)
    if loglevel>=1:
        print("base flags generated")
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    if loglevel>=1:
        print("avals constructed")
    
    #
    # create the table data
    #
    
    ftype_flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in self.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in ftype_flags]
    if loglevel>=1:
        print("ftypes constructed")
    flags = [self.generate_flags((target_size + ftype.size())//2, ftype) for ftype in ftypes]
    flag_sizes = [len(xx) for xx in flags]
    if loglevel>=1:
        print("typed flags constructed, their length is {}".format(flag_sizes))
    
    sym_asym_mats = [sym_asym_bases(self, (target_size + ftype.size())//2, ftype) for ftype in ftypes]
    if loglevel>=1:
        print("sym and asym matrices constructed")
    
    table_constructor = {}
    for ii, ftype in (pbar := tqdm(enumerate(ftypes))):
        ns = (target_size + ftype.size())//2
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, [])
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        table_constructor[(ns, ftype)] = bases
        pbar.set_description("done with mult table for {}".format(ftype))
    if loglevel>=1:
        print("tables constructed")
    
    sdp_data = tables_to_sdp_data(self, table_constructor)
    if loglevel>=1:
        print("tables added to sdp data")
    
    #
    # add constraints data
    #
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
            if loglevel>=1:
                print("done with constraint for \n{}\n".format(fv))
        constraints_vals = [0]*len(constraints_flags)
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    positives_list_exact = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    positives_matrix_exact = matrix(positives_list_exact)
    if loglevel>=1:
        print("constraints constructed")
    
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    if loglevel>=1:
        print("constraints added to sdp data")
    
    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if (not rounding):
        if loglevel>=1:
            print("running sdp without construction")
        
        mat_inds, mat_vals, block_sizes = sdp_data
    
        time.sleep(float(0.1))
        initial_sol = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
        time.sleep(float(0.1))
        
        phi_vector_original = initial_sol['y']
        phi_vector_rounded = cfr_adaptive(initial_sol['y'], one_vector.values())
        if loglevel>=1:
            print("rounded phi vector is: \n{}".format(print_sparse(phi_vector_rounded)))
        if certificate:
            return initial_sol
        else:
            return initial_sol['primal']

    if isinstance(construction, Flag):
        phi_vectors_exact = [construction.values()]
    else:
        phi_vectors_exact = [xx.values() for xx in construction]
    
    #
    # adjust the table to consider the kernel from y_rounded
    #
    
    table_constructor = adjust_table_phi(self, table_constructor, phi_vectors_exact)
    sdp_data = tables_to_sdp_data(self, table_constructor)
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data
    
    if loglevel>=1:
        print("running SDP after kernel correction with block sizes {}".format(block_sizes))
    
    time.sleep(float(0.1))
    final_sdp = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
    time.sleep(float(0.1))
    
    res = round_sdp_solution(self, final_sdp, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact)

    for ii, Xii in enumerate(res[1]):
        if block_sizes[ii]<=0:
            continue
        X_eigenvalues = matrix(Xii).eigenvalues()
        print_debug(X_eigenvalues, "\n\n")
    
    if maximize:
        res = (-res[0], res[1])
    if certificate:
        return res
    else:
        return res[0]

In [80]:
import itertools

# a general identifier code that works on any number of colors and edge arity
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    ftype_union = [jj for ff in ftype_points for jj in ff]
    g_parts = list(ftype_points) + \
              [[ii for ii in range(n) if ii not in ftype_union]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# a general generator code that also works on any number of colors and edge arity
def _generateCT(k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    if k==2:
        BT = GraphTheory
    if k==3:
        BT = ThreeGraphTheory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

#to make a resulting theory create the functions directly with the correct number of parameters:

#for example, to make graphs with 2 unordered colors
#we say k=2 to talk about regular graphs
#and order_partition=[[0, 1]] to say that 0 and 1 can be interchanged (if you want ordered colors then the partition is [[0], [1]])
#edges, C0, C1 are the relations
def test_identify(n, ftype_points, edges, C0, C1):
    return _identifyCT(2, [[0, 1]], n, ftype_points, edges=edges, C0=C0, C1=C1)

#same for the generator. k=2 and order_partition=[[0, 1]].
def test_generate(n):
    return _generateCT(2, [[0, 1]], n)

TTT = CombinatorialTheory("2UnOrdColGraph", test_generate, test_identify, edges=2, C0=1, C1=1)

tri_000 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1], [2]])
tri_001 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1]], C1=[[2]])
TTT.exclude([tri_000, tri_001])

Bp = TTT(2, edges=[[0, 1]], C0=[[0], [1]], ftype=[0])
Gp = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]], ftype=[0])

G = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]])
B = TTT(2, edges=[[0, 1]], C0=[[0], [1]])
M = TTT(2, C0=[[0]], C1=[[1]])

optim = blowup_construction(TTT, 4, 2, edges=[[0, 1]], C0=[[0]], C1=[[1]], symmetric=True, variable=True)
ssoptim = set_sum(optim)
der_optims = derivatives(ssoptim, [1/2])

loglevel=1
optimize_problem(TTT, B - M, 4, positives=[Gp-Bp, G-1/3], certificate=False, construction=der_optims)

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 142.25it/s]


[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
base flags generated
avals constructed
ftypes constructed
typed flags constructed, their length is [8, 8, 6, 6]
sym and asym matrices constructed


done with mult table for Ftype on 2 points with edges=[[0, 1]], C0=[[0]], C1=[[1]]: : 4it [00:00, 545.74it/s]


tables constructed
tables added to sdp data
done with constraint for 
Flag Algebra Element over Rational Field
0  - Flag on 2 points, ftype from [0] with edges=[], C0=[[0], [1]], C1=[]
0  - Flag on 2 points, ftype from [0] with edges=[], C0=[[0]], C1=[[1]]
-1 - Flag on 2 points, ftype from [0] with edges=[[0, 1]], C0=[[0], [1]], C1=[]
1  - Flag on 2 points, ftype from [0] with edges=[[0, 1]], C0=[[0]], C1=[[1]]

done with constraint for 
Flag Algebra Element over Rational Field
-1/3 - Flag on 2 points, ftype from [] with edges=[], C0=[[0], [1]], C1=[]
-1/3 - Flag on 2 points, ftype from [] with edges=[], C0=[[0]], C1=[[1]]
-1/3 - Flag on 2 points, ftype from [] with edges=[[0, 1]], C0=[[0], [1]], C1=[]
2/3  - Flag on 2 points, ftype from [] with edges=[[0, 1]], C0=[[0]], C1=[[1]]

constraints constructed
constraints added to sdp data
running SDP after kernel correction with block sizes [4, 2, 4, 4, 4, 2, 2, 2, -32, -23]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00

0

In [77]:
res

(0,
 [
[ 355/1024  -17/1024    15/512 -141/1024]                       
[ -17/1024  165/1024 -167/1024    5/1024]                       
[   15/512 -167/1024   125/512   -9/1024]  [  153/256  915/1024]
[-141/1024    5/1024   -9/1024  287/1024], [ 915/1024 1491/1024],

[ 741/1024    51/512   121/512 -547/1024]
[   51/512  107/1024   25/1024 -149/1024]
[  121/512   25/1024   277/512 -165/1024]
[-547/1024 -149/1024 -165/1024     31/64],

[    3/16   53/256  -5/1024  -53/512]
[  53/256  209/256   11/512 -153/256]
[ -5/1024   11/512 217/1024 -21/1024]
[ -53/512 -153/256 -21/1024  137/256],

[  29/1024    -1/512   -21/512   15/1024]                       
[   -1/512      5/16         0   11/1024]                       
[  -21/512         0     11/32 -253/1024]  [  173/256   605/512]
[  15/1024   11/1024 -253/1024  241/1024], [  605/512 2363/1024],

[173/512 -33/512]  [311/1024 -113/512]                                                                                                           

In [69]:
optim = blowup_construction(TTT, 4, 2, edges=[[0, 1]], C0=[[0]], C1=[[1]], symmetric=True, variable=True)
ssoptim = set_sum(optim)
der_optims = derivatives(ssoptim, [1/2])
der_optims

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 115.77it/s]


[Flag Algebra Element over Multivariate Polynomial Ring in X0, X1 over Rational Field
 1/8 - Flag on 4 points, ftype from [] with edges=[], C0=[[0], [1], [2], [3]], C1=[]
 1/2 - Flag on 4 points, ftype from [] with edges=[[0, 3], [1, 3], [2, 3]], C0=[[0], [1], [2]], C1=[[3]]
 3/8 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 2], [1, 3]], C0=[[0], [1]], C1=[[2], [3]],
 Flag Algebra Element over Multivariate Polynomial Ring in X0, X1 over Rational Field
 6  - Flag on 4 points, ftype from [] with edges=[], C0=[[0], [1], [2], [3]], C1=[]
 -6 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 2], [1, 3]], C0=[[0], [1]], C1=[[2], [3]],
 Flag Algebra Element over Multivariate Polynomial Ring in X0, X1 over Rational Field
 48   - Flag on 4 points, ftype from [] with edges=[], C0=[[0], [1], [2], [3]], C1=[]
 -192 - Flag on 4 points, ftype from [] with edges=[[0, 3], [1, 3], [2, 3]], C0=[[0], [1], [2]], C1=[[3]]
 144  - Flag on 4 points, ftype from [] with edge